In [11]:
from google.colab import files

# Upload faqs.json
uploaded = files.upload()


Saving faqs.json to faqs (3).json


In [12]:
!pip install fastapi uvicorn fuzzywuzzy python-Levenshtein spacy pyngrok transformers torch
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import BertTokenizer, BertModel
import torch
import json
from sklearn.metrics.pairwise import cosine_similarity

# Load the Sentence-BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = BertModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Load FAQ data from the uploaded JSON file
with open("/content/faqs.json") as f:  # Use the correct path here
    faq_data = json.load(f)

# Convert FAQ data into a flat list and compute embeddings
faq_list = []
faq_embeddings = []

for category, entries in faq_data.items():
    for entry in entries:
        faq_list.append({
            "category": category,
            "question": entry["question"],
            "answer": entry["answer"]
        })

        # Compute embeddings for each question
        inputs = tokenizer(entry["question"], return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # Get the average of the last hidden state
            faq_embeddings.append(embeddings.squeeze())  # Use squeeze to flatten the tensor

faq_embeddings = torch.stack(faq_embeddings)  # Convert list to tensor

app = FastAPI()

# Allow CORS for all origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Change to ["http://localhost:3000"] if running locally
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class Query(BaseModel):
    query: str

def compute_embedding(query):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze()  # Get the average of the last hidden state and flatten
    return embedding

@app.post("/search")
async def search_faq(query: Query):
    query_embedding = compute_embedding(query.query)

    # Calculate cosine similarity between the query embedding and all FAQ embeddings
    similarities = cosine_similarity(query_embedding.numpy().reshape(1, -1), faq_embeddings.numpy())

    # Find the index of the highest similarity score
    best_match_index = similarities.argmax()

    # Return the corresponding FAQ
    best_faq = faq_list[best_match_index]

    return {
        "question": best_faq["question"],
        "answer": best_faq["answer"],
        "category": best_faq["category"]
    }


In [15]:
!ngrok authtoken 2nUYAVrRHFh3iZveHjqBhEiRpBJ_5bvSWQwCDU5M3m8kwX1JX

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Change to a different port
ngrok_port = 8001

# Start ngrok to tunnel the FastAPI server
ngrok_tunnel = ngrok.connect(ngrok_port)
print("Public URL:", ngrok_tunnel.public_url)

# Run the FastAPI app using asyncio
nest_asyncio.apply()  # Allows us to run the server in the notebook environment

# Start the FastAPI app on a different port
uvicorn.run(app, host="0.0.0.0", port=ngrok_port)


Public URL: https://e47d-34-121-2-35.ngrok-free.app


INFO:     Started server process [3957]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     103.21.125.86:0 - "OPTIONS /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.21.125.86:0 - "POST /search HTTP/1.1" 2